In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#customer_most_channel_sales['client_id'] = customer_most_channel_sales['client_id'].astype(int)

In [4]:
df = pd.read_parquet('/content/drive/MyDrive/Elective_Groups/Capstone/Step_2_transactions_dataset_outlier_removal.parquet')
df['client_id'] = df['client_id'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26061261 entries, 0 to 26061260
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date_order     datetime64[ns]
 1   product_id     category      
 2   client_id      int64         
 3   sales_net      float32       
 4   quantity       int32         
 5   order_channel  category      
 6   branch_id      category      
 7   month          object        
dtypes: category(3), datetime64[ns](1), float32(1), int32(1), int64(1), object(1)
memory usage: 990.1+ MB


#### Main Channel used for purchases. (Monetary)

In [5]:
channel_sales = df[['client_id','order_channel','sales_net']].groupby(['client_id','order_channel']).sum().reset_index()
channel_sales = channel_sales.loc[channel_sales.sales_net != 0]
channel_sales.head()

,client_id,order_channel,sales_net
0,6,at the store,1318.594360
1,6,by phone,4563.638672
5,7,at the store,1190.777100
10,25,at the store,374.586090
15,37,at the store,2356.573242


In [6]:
channel_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271372 entries, 0 to 743621
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   client_id      271372 non-null  int64   
 1   order_channel  271372 non-null  category
 2   sales_net      271372 non-null  float32 
dtypes: category(1), float32(1), int64(1)
memory usage: 5.4 MB


In [7]:
customer_channel_sales = channel_sales.groupby('client_id').max().reset_index()
customer_channel_sales['max'] = 1
customer_channel_sales.head()

<ipython-input-7-4a20c9fd315a>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  customer_channel_sales = channel_sales.groupby('client_id').max().reset_index()


,client_id,sales_net,max
0,6,4563.638672,1
1,7,1190.777100,1
2,25,374.586090,1
3,37,4234.562988,1
4,40,20465.156250,1


In [8]:
customer_most_channel_sales = channel_sales.merge(customer_channel_sales,on=['client_id','sales_net'])
customer_most_channel_sales

,client_id,order_channel,sales_net,max
0,6,by phone,4563.638672,1
1,7,at the store,1190.777100,1
2,25,at the store,374.586090,1
3,37,by phone,4234.562988,1
4,40,by phone,20465.156250,1
...,...,...,...,...
147034,2274440,by phone,14371.222656,1
147035,2274456,at the store,1422.021484,1
147036,2274479,at the store,418.696594,1
147037,2274504,at the store,88.485603,1


#### Most Recent channel used?

In [9]:
df.head()

,date_order,product_id,client_id,sales_net,quantity,order_channel,branch_id,month
0,2017-09-25,2376333,188502,155.443207,3,online,2732,2017-09
1,2017-09-25,2536756,1977896,365.766235,21,online,6184,2017-09
2,2017-09-25,203377,2086861,1093.374023,3,by phone,7051,2017-09
3,2017-09-25,745693,482822,1.449000,5,by phone,2771,2017-09
4,2017-09-25,683924,224223,12.033600,3,at the store,9463,2017-09


In [10]:
date_order_chanell = df[['client_id','order_channel','date_order']].groupby(['client_id','order_channel']).max().reset_index()
date_order_chanell.head()

,client_id,order_channel,date_order
0,6,at the store,2019-07-31
1,6,by phone,2019-08-13
2,6,during the visit of a sales rep,NaT
3,6,online,NaT
4,6,other,NaT


In [11]:
max_date_order_chanell = customer_most_channel_sales.merge(date_order_chanell,on=['client_id','order_channel'],how='left')
max_date_order_chanell

,client_id,order_channel,sales_net,max,date_order
0,6,by phone,4563.638672,1,2019-08-13
1,7,at the store,1190.777100,1,2018-12-24
2,25,at the store,374.586090,1,2018-10-10
3,37,by phone,4234.562988,1,2018-05-21
4,40,by phone,20465.156250,1,2019-09-02
...,...,...,...,...,...
147034,2274440,by phone,14371.222656,1,2019-09-02
147035,2274456,at the store,1422.021484,1,2019-09-02
147036,2274479,at the store,418.696594,1,2019-02-27
147037,2274504,at the store,88.485603,1,2017-11-02


#### #column simply stating if the customer went through a refund (1,0)

In [12]:
sales_refund = df.loc[df.sales_net < 0]
sales_refund = sales_refund['client_id'].drop_duplicates().to_frame()
sales_refund['refund_flag'] = 1
sales_refund.head()

,client_id,refund_flag
92,1791722,1
464,1324117,1
614,2018979,1
1090,624514,1
1153,1760477,1


#### Can also create three general groups based off the score of the RFM. I have the code for this and could use your help to work out why my clustering doesn't work as i want to check what would be the ideal amount of k means clusters would be.

In [13]:
customer_dash = pd.read_csv('/content/drive/MyDrive/Elective_Groups/Capstone/Step_5_Dashboard.csv')
customer_dash.head()

,Unnamed: 0,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,...,Other_Average_Sales,churn_assumption1,Recency,Frequency,Monetary_Value,Recency_Quantitle,Frequency_Quantitle,Monetary_Value_Quantitle,RFM,RFM_Segment
0,0,188502,140138,227.496753,616,66700.336,108.279770,42,5.250000,8,...,0.1656,False,17,5.819672,65646.055,4,5,5,455,Champion
1,1,1977896,293627,567.943907,517,145390.900,281.220340,260,43.333333,6,...,NaN,True,321,4.951220,143088.060,1,5,5,155,Cannot Lose Them
2,2,2086861,19161,58.956923,325,40332.010,124.098495,2018,504.500000,4,...,NaN,False,10,6.894231,38954.150,5,5,5,555,Champion
3,3,482822,6740,37.444444,180,16716.584,92.869910,0,NaN,0,...,NaN,False,16,16.534884,16716.584,4,4,4,444,Loyal Customer
4,4,224223,27219,26.096836,1043,124999.440,119.846054,105,8.076923,13,...,NaN,False,2,3.625000,119716.470,5,5,5,555,Champion


In [14]:
customer_dash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148725 entries, 0 to 148724
Data columns (total 53 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         148725 non-null  int64  
 1   client_id                          148725 non-null  int64  
 2   Total_Quantity                     148725 non-null  int64  
 3   Average_Quantity                   148725 non-null  float64
 4   Number_of_Purchase                 148725 non-null  int64  
 5   Total_Sales                        148725 non-null  float64
 6   Average_Sales                      148725 non-null  float64
 7   R_Total_Quantity                   148725 non-null  int64  
 8   R_Average_Quantity                 88128 non-null   float64
 9   R_Number_of_Purchase               148725 non-null  int64  
 10  R_Total_Sales                      148725 non-null  float64
 11  R_Average_Sales                    8812

In [15]:
customer_dash['RFM_Score'] = customer_dash['Recency_Quantitle'] + customer_dash['Frequency_Quantitle'] + customer_dash['Monetary_Value_Quantitle']

In [16]:
customer_dash.head()

,Unnamed: 0,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,...,churn_assumption1,Recency,Frequency,Monetary_Value,Recency_Quantitle,Frequency_Quantitle,Monetary_Value_Quantitle,RFM,RFM_Segment,RFM_Score
0,0,188502,140138,227.496753,616,66700.336,108.279770,42,5.250000,8,...,False,17,5.819672,65646.055,4,5,5,455,Champion,14
1,1,1977896,293627,567.943907,517,145390.900,281.220340,260,43.333333,6,...,True,321,4.951220,143088.060,1,5,5,155,Cannot Lose Them,11
2,2,2086861,19161,58.956923,325,40332.010,124.098495,2018,504.500000,4,...,False,10,6.894231,38954.150,5,5,5,555,Champion,15
3,3,482822,6740,37.444444,180,16716.584,92.869910,0,NaN,0,...,False,16,16.534884,16716.584,4,4,4,444,Loyal Customer,12
4,4,224223,27219,26.096836,1043,124999.440,119.846054,105,8.076923,13,...,False,2,3.625000,119716.470,5,5,5,555,Champion,15


In [17]:
def segments(df):
    if df['RFM_Score'] > 10 :
        return 'Gold'
    elif (df['RFM_Score'] > 5) and (df['RFM_Score'] <= 10 ):
        return 'Silver'
    else:
        return 'Bronze'
customer_dash['General_Segment'] = customer_dash.apply(lambda x: segments(x), axis=1)
customer_dash.head()

,Unnamed: 0,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,...,Recency,Frequency,Monetary_Value,Recency_Quantitle,Frequency_Quantitle,Monetary_Value_Quantitle,RFM,RFM_Segment,RFM_Score,General_Segment
0,0,188502,140138,227.496753,616,66700.336,108.279770,42,5.250000,8,...,17,5.819672,65646.055,4,5,5,455,Champion,14,Gold
1,1,1977896,293627,567.943907,517,145390.900,281.220340,260,43.333333,6,...,321,4.951220,143088.060,1,5,5,155,Cannot Lose Them,11,Gold
2,2,2086861,19161,58.956923,325,40332.010,124.098495,2018,504.500000,4,...,10,6.894231,38954.150,5,5,5,555,Champion,15,Gold
3,3,482822,6740,37.444444,180,16716.584,92.869910,0,NaN,0,...,16,16.534884,16716.584,4,4,4,444,Loyal Customer,12,Gold
4,4,224223,27219,26.096836,1043,124999.440,119.846054,105,8.076923,13,...,2,3.625000,119716.470,5,5,5,555,Champion,15,Gold


In [18]:
customer_dash.groupby('General_Segment').agg({'Recency':'mean','Frequency':'mean','Monetary_Value':['mean','count']}).round(1).reset_index()

General_Segment Recency Frequency Monetary_Value       
                     mean      mean           mean  count
0          Bronze   323.0     222.7          754.0  28809
1            Gold    25.8      12.2        63359.4  52164
2          Silver   149.5      54.3         6991.5  67752

In [19]:
customer_dash[['client_id','RFM_Score','General_Segment']]

,client_id,RFM_Score,General_Segment
0,188502,14,Gold
1,1977896,11,Gold
2,2086861,15,Gold
3,482822,12,Gold
4,224223,15,Gold
...,...,...,...
148720,144085,10,Silver
148721,450600,5,Bronze
148722,1511299,6,Silver
148723,923579,6,Silver


#### Merge with the data from Cohort Analysis for long term engaged customers.


In [20]:
ltcust_df = pd.read_csv('/content/drive/MyDrive/Elective_Groups/Capstone/Longterm_interacting_cust.csv')
ltcust_df["Longterm_Engaged"]=1
ltcust_df.head()

,Unnamed: 0,client_id,Longterm_Engaged
0,0,1172710,1
1,1,749713,1
2,2,268229,1
3,3,1718077,1
4,4,1620852,1


In [21]:
df_merged = customer_dash.merge(ltcust_df[['client_id', 'Longterm_Engaged']], on='client_id', how='left', suffixes=('_left', '_right'))
df_merged['Longterm_Engaged'] = df_merged['Longterm_Engaged'].fillna(0)
df_merged = df_merged.drop('Unnamed: 0', axis =1)
df_merged.head()

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,...,Frequency,Monetary_Value,Recency_Quantitle,Frequency_Quantitle,Monetary_Value_Quantitle,RFM,RFM_Segment,RFM_Score,General_Segment,Longterm_Engaged
0,188502,140138,227.496753,616,66700.336,108.279770,42,5.250000,8,-1054.2786,...,5.819672,65646.055,4,5,5,455,Champion,14,Gold,1.0
1,1977896,293627,567.943907,517,145390.900,281.220340,260,43.333333,6,-2302.8389,...,4.951220,143088.060,1,5,5,155,Cannot Lose Them,11,Gold,0.0
2,2086861,19161,58.956923,325,40332.010,124.098495,2018,504.500000,4,-1377.8613,...,6.894231,38954.150,5,5,5,555,Champion,15,Gold,1.0
3,482822,6740,37.444444,180,16716.584,92.869910,0,NaN,0,0.0000,...,16.534884,16716.584,4,4,4,444,Loyal Customer,12,Gold,1.0
4,224223,27219,26.096836,1043,124999.440,119.846054,105,8.076923,13,-5282.9673,...,3.625000,119716.470,5,5,5,555,Champion,15,Gold,1.0


#### Merge with the data from main channels.


In [22]:
customer_most_channel_sales.head()

,client_id,order_channel,sales_net,max
0,6,by phone,4563.638672,1
1,7,at the store,1190.777100,1
2,25,at the store,374.586090,1
3,37,by phone,4234.562988,1
4,40,by phone,20465.156250,1


In [23]:
customer_most_channel_sales.rename(columns={'order_channel': 'Main_channel', 'sales_net': 'Main_channel_amount'}, inplace=True)
customer_most_channel_sales

,client_id,Main_channel,Main_channel_amount,max
0,6,by phone,4563.638672,1
1,7,at the store,1190.777100,1
2,25,at the store,374.586090,1
3,37,by phone,4234.562988,1
4,40,by phone,20465.156250,1
...,...,...,...,...
147034,2274440,by phone,14371.222656,1
147035,2274456,at the store,1422.021484,1
147036,2274479,at the store,418.696594,1
147037,2274504,at the store,88.485603,1


In [24]:
df_merged.loc[df_merged.client_id == 1000078]

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,...,Frequency,Monetary_Value,Recency_Quantitle,Frequency_Quantitle,Monetary_Value_Quantitle,RFM,RFM_Segment,RFM_Score,General_Segment,Longterm_Engaged
9267,1000078,13361,86.2,155,30274.775,195.32114,3235,82.948718,39,-9034.761,...,45.083333,21240.016,2,2,4,224,At Risk,8,Silver,0.0


In [25]:
df_merged = df_merged.merge(customer_most_channel_sales[['client_id', 'Main_channel', 'Main_channel_amount']], 
                            on='client_id', 
                            how='left')
df_merged.head()

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,...,Recency_Quantitle,Frequency_Quantitle,Monetary_Value_Quantitle,RFM,RFM_Segment,RFM_Score,General_Segment,Longterm_Engaged,Main_channel,Main_channel_amount
0,188502,140138,227.496753,616,66700.336,108.279770,42,5.250000,8,-1054.2786,...,4,5,5,455,Champion,14,Gold,1.0,online,50733.843750
1,1977896,293627,567.943907,517,145390.900,281.220340,260,43.333333,6,-2302.8389,...,1,5,5,155,Cannot Lose Them,11,Gold,0.0,online,135610.687500
2,2086861,19161,58.956923,325,40332.010,124.098495,2018,504.500000,4,-1377.8613,...,5,5,5,555,Champion,15,Gold,1.0,by phone,26455.498047
3,482822,6740,37.444444,180,16716.584,92.869910,0,NaN,0,0.0000,...,4,4,4,444,Loyal Customer,12,Gold,1.0,by phone,9846.728516
4,224223,27219,26.096836,1043,124999.440,119.846054,105,8.076923,13,-5282.9673,...,5,5,5,555,Champion,15,Gold,1.0,at the store,66322.343750


#### Add Product Information for Clients


In [26]:
client_product_db = pd.read_csv('/content/drive/MyDrive/Elective_Groups/Capstone/client_product_db.csv')
client_product_db.head()


,Unnamed: 0,client_id,product_id,sales_net,product_type_sum
0,0,2274517,1452150,3772.666818,Product_A
1,1,2274508,1253238,0.000000,Product_A
2,2,2274504,1146070,53.902800,Product_A
3,3,2274479,462354,121.605600,Product_A
4,4,2274456,1298661,342.198907,Product_A


In [27]:
df_merged = df_merged.merge(client_product_db[['client_id', 'product_id', 'sales_net','product_type_sum']], 
                            on='client_id', 
                            how='left')

In [28]:
df_merged.rename(columns={'product_id': 'Main_product', 'sales_net': 'Main_product_amount','product_type_sum': 'product_group'}, inplace=True)
df_merged.head()

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,...,RFM,RFM_Segment,RFM_Score,General_Segment,Longterm_Engaged,Main_channel,Main_channel_amount,Main_product,Main_product_amount,product_group
0,188502,140138,227.496753,616,66700.336,108.279770,42,5.250000,8,-1054.2786,...,455,Champion,14,Gold,1.0,online,50733.843750,603987,8335.200000,Product_A
1,1977896,293627,567.943907,517,145390.900,281.220340,260,43.333333,6,-2302.8389,...,155,Cannot Lose Them,11,Gold,0.0,online,135610.687500,130187,17327.610543,Product_A
2,2086861,19161,58.956923,325,40332.010,124.098495,2018,504.500000,4,-1377.8613,...,555,Champion,15,Gold,1.0,by phone,26455.498047,1119307,3209.456800,Product_A
3,482822,6740,37.444444,180,16716.584,92.869910,0,NaN,0,0.0000,...,444,Loyal Customer,12,Gold,1.0,by phone,9846.728516,358838,1765.654800,Product_A
4,224223,27219,26.096836,1043,124999.440,119.846054,105,8.076923,13,-5282.9673,...,555,Champion,15,Gold,1.0,at the store,66322.343750,840202,4816.200000,Product_A


#### Add Sporadic RFM Segment. Add Refund Column


In [29]:
#issue
#df_merged['First_Activity_Month'].strftime('%Y-%m')
#a.strftime('%b. %d, %Y was a %A')

In [30]:
pd.to_datetime(df_merged['First_Activity_Month']).dt.strftime('%Y')

0         2017
1         2017
2         2017
3         2017
4         2017
          ... 
148731    2019
148732    2018
148733    2019
148734    2019
148735    2019
Name: First_Activity_Month, Length: 148736, dtype: object

In [31]:

df_merged['refund_flag'] = (df_merged['R_Total_Quantity'] > 0).astype(int)

# Define the conditions
condition = (df_merged['RFM_Segment'] == 'New Customer') & (pd.to_datetime(df_merged['First_Activity_Month']).dt.strftime('%Y') != pd.to_datetime(df_merged['Last_Activity_Month']).dt.strftime('%Y'))

# Replace the values of the RFM_SEGMENTS column
df_merged.loc[condition, 'RFM_Segment'] = 'Esporadic'


df_merged.to_csv('/content/drive/MyDrive/Elective_Groups/Capstone/Step_6_Dashboard.csv')


#


In [32]:
df_merged.loc[df_merged.Frequency.isna()]

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,...,RFM_Segment,RFM_Score,General_Segment,Longterm_Engaged,Main_channel,Main_channel_amount,Main_product,Main_product_amount,product_group,refund_flag
